In [1]:
import json
import re
import threading
import traceback
import requests
from fastapi import HTTPException
from openai import OpenAI

In [2]:
SERPER_API = "8fa4e2d156e1822e2342a83a5455c9a44abd736d"
DEEPSEEK_API = "sk-ef862e2707dc49ae868be865043e7da7"
LEPTON_API = "EcH9KDJ856w2EZFzEp2pGkdLIWCESrj2"
GROQ_API = "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"
BING_SEARCH_V7_ENDPOINT = "https://api.bing.microsoft.com/v7.0/search"
SEARCHAPI_SEARCH_ENDPOINT = "https://www.searchapi.io/api/v1/search"
SERPER_SEARCH_ENDPOINT = "https://google.serper.dev/search"
GOOGLE_SEARCH_ENDPOINT = "https://customsearch.googleapis.com/customsearch/v1"
REFERENCE_COUNT = 8
DEFAULT_SEARCH_ENGINE_TIMEOUT = 10

In [3]:
_rag_query_text = """
You are a large language AI assistant built by Group 14. You are given a user question, and please write clean, concise and accurate answer to the question. You will be given a set of related contexts to the question, each starting with a reference number like [[citation:x]], where x is a number. Please use the context and cite the context at the end of each sentence if applicable.

Your answer must be correct, accurate and written by an expert using an unbiased and professional tone. Please limit to 2048 tokens. Do not give any information that is not related to the question, and do not repeat. Say "information is missing on" followed by the related topic, if the given context do not provide sufficient information.

Please cite the contexts with the reference numbers, in the format [citation:x]. If a sentence comes from multiple contexts, please list all applicable citations, like [citation:3][citation:5]. Other than code and specific names and citations, your answer must be written in the same language as the question. If there are too many citations, choose the best of them
Here are the set of contexts:

{context}

Remember, don't blindly repeat the contexts. And here is the user question:
"""
stop_words = [
    "<|im_end|>",
    "[End]",
    "[end]",
    "\nReferences:\n",
    "\nSources:\n",
    "End.",
]

_more_questions_prompt = """
You are a helpful assistant that helps the user to ask related questions, based on user's original question and the related contexts. Please identify worthwhile topics that can be follow-ups, and write questions no longer than 20 words each. Please make sure that specifics, like events, names, locations, are included in follow up questions so they can be asked standalone. For example, if the original question asks about "the Manhattan project", in the follow up question, do not just say "the project", but use the full name "the Manhattan project". The format of giving the responses and generating the questions shoudld be like this:

1. [Question 1]
2. [Question 2] 
3. [Question 3]

Here are the contexts of the question:

{context}

Remember, based on the original question and related contexts, suggest three such further questions. Do NOT repeat the original question. Each related question should be no longer than 20 words. Here is the original question:
"""


In [4]:
def search_with_serper(query: str, subscription_key=SERPER_API, prints=False):
    """
    Search with serper and return the contexts.
    """
    payload = json.dumps({
        "q": query,
        "num": (
            REFERENCE_COUNT
            if REFERENCE_COUNT % 10 == 0
            else (REFERENCE_COUNT // 10 + 1) * 10
        ),
    })
    headers = {"X-API-KEY": subscription_key, "Content-Type": "application/json"}
    response = requests.post(
        SERPER_SEARCH_ENDPOINT,
        headers=headers,
        data=payload,
        timeout=DEFAULT_SEARCH_ENGINE_TIMEOUT,
    )
    if not response.ok:
        raise HTTPException(response.status_code, "Search engine error.")
    json_content = response.json()

    if prints:
        print(json_content)
        print("\n\n\n-------------------------------------------------------------------------------\n\n\n")

    try:
        contexts = []
        if json_content.get("knowledgeGraph"):
            url = json_content["knowledgeGraph"].get("descriptionUrl") or json_content["knowledgeGraph"].get("website")
            snippet = json_content["knowledgeGraph"].get("description")
            if url and snippet:
                contexts.append({
                    "name": json_content["knowledgeGraph"].get("title",""),
                    "url": url,
                    "snippet": snippet
                })
        if json_content.get("answerBox"):
            url = json_content["answerBox"].get("url")
            snippet = json_content["answerBox"].get("snippet") or json_content["answerBox"].get("answer")
            if url and snippet:
                contexts.append({
                    "name": json_content["answerBox"].get("title",""),
                    "url": url,
                    "snippet": snippet
                })
        contexts += [
            {"name": c["title"], "url": c["link"], "snippet": c.get("snippet","")}
            for c in json_content["organic"]
        ]

        if prints:
            print(contexts[:REFERENCE_COUNT])
        return contexts[:REFERENCE_COUNT]
    
    except KeyError:
        return []

In [5]:
def extract_citation_numbers(sentence):
    pattern = r'\[citation:(\d+)\]'

    citation_numbers = re.findall(pattern, sentence)

    return citation_numbers

In [6]:
def fetch_json_attributes(json_data, print=False):
    
    names = []
    urls = []
    snippets = []

    for item in json_data:
        names.append(item['name'])
        urls.append(item['url'])
        snippets.append(item['snippet'])

    if print:
        print("Names:", names)
        print("URLs:", urls)
        print("Snippets:", snippets)

    return names, urls, snippets

In [7]:
class AI():

    def DeepSeek(system_prompt, query):
        client = OpenAI(
            api_key=DEEPSEEK_API, 
            base_url="https://api.deepseek.com/v1")
        
        llm_response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": query},
                    ],
                    max_tokens=1024,
                    stop=stop_words,
                    stream=True,
                    temperature=0.9,
                ) 
        

        chunks = []
        for chunk in llm_response:
            if chunk.choices[0].delta.content is not None:
                print(chunk.choices[0].delta.content, end="")
                chunks.append(chunk.choices[0].delta.content)


        print("\n\n")
        complete_response = ''.join(chunks)

        return complete_response
    
    def Groq(system_prompt, query):

        client = OpenAI(
            base_url = "https://api.groq.com/openai/v1",
            api_key=GROQ_API
            )
        llm_response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": query
                }
            ],
            temperature=0.5,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )

        chunks = []
        for chunk in llm_response:    
            try:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
                    chunks.append(chunk.choices[0].delta.content)
            except:
                pass


        print("\n\n")
        complete_response = ''.join(chunks)

        return complete_response

In [40]:
def get_related_questions(query, contexts):
        
        system_prompt = _more_questions_prompt.format(
                            context="\n\n".join([c["snippet"] for c in contexts])
                        )

        try:
            complete_response = AI.Groq(system_prompt, query)
            return complete_response
        
        except Exception as e:
            print(e)
            return []

In [41]:
def generate_answer(query, contexts):
    query = re.sub(r"\[/?INST\]", "", query)

    system_prompt = _rag_query_text.format(
                context="\n\n".join(
                    [f"[[citation:{i+1}]] {c['snippet']}" for i, c in enumerate(contexts)]
                )
            )

    try:
        complete_response = AI.Groq(system_prompt, query)
        return complete_response

    except Exception as e:
        print(e)
        return "Failed Response"
    

In [42]:
def main(query, contexts, urls):

    print("Sources ---->")
    for _url in urls:
        print(_url)

    print("\n\nAnswers --->")
    citations = extract_citation_numbers(generate_answer(query, contexts))
    print('\n'.join([f"Citation : {citation} --->  {urls[int(citation)-1]}" for citation in citations]))

                
    print("\n\nRelated Questions --->")
    get_related_questions(query, contexts)

In [43]:
query = "tell me something about Halo"
contexts = search_with_serper(query)
name, url, snippets = fetch_json_attributes(contexts)

In [44]:
main(query, contexts, url)

Sources ---->
https://en.wikipedia.org/wiki/Halo_(franchise)
https://www.youtube.com/watch?v=4he5CUyNGwM
https://www.reddit.com/r/HaloStory/comments/u5e6pu/what_the_halo_show_is_about/
https://www.vice.com/en/article/the-complete-untold-history-of-halo-an-oral-history/
https://www.quora.com/How-do-I-explain-halo-the-game-without-spoilers-As-in-to-convince-someone-to-play-that-has-never-played-before
https://www.youtube.com/watch?v=Gs280MSsLN0
https://www.quora.com/How-would-you-describe-Halo
https://www.youtube.com/watch?v=0vQ1q1msPXI


Answers --->
Halo is a military science fiction video game series and media franchise that originated from a strategic position, rather than being mapped as a shooter from the outset [citation:4]. It has evolved to become a popular franchise, currently managed and developed by Halo Studios [citation:1]. The series is known for its engaging gameplay, with elements of a "shoot em' up" type of game, featuring an awesome campaign and other essential compone